# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: CHRISTIAN MAURICIO YEPEZ ESCALANTE  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:






                                        

#Modelo
**¿Como represento el espacio de soluciones?**

Las soluciones se las representara de la siguiente manera con las siguientes variables:

* Tomas: Cada toma de la película se identifica de manera única (por ejemplo, $T_1$, $T_2$, ..., $T_n$).
* Actores: Cada actor de doblaje se identifica de manera única (por ejemplo, $A_1$, $A_2$, ..., $A_m$).
* Días: Los días necesarios para completar el doblaje se identifican secuencialmente ($D_1$, $D_2$, ..., $D_k$), donde k es el número máximo de días necesarios.

Los datos de entrada se los representan como una una matriz donde las filas representan las tomas y las columnas representan a los actores. Cada entrada de la matriz indica si el actor participa (1) o no (0) en esa toma. Esto se lo transforma a un diccionario en el cual cada toma  es la clave y tiene asociado un vector de cada una de los actores que son requeridos para esta.

Finalmente se tiene como salida dos diccionarios de asociados a los días de las tomas, uno con las tomas que se harán ese día y otro con los actores que son convocados para las tomas.

**¿Cual es la función objetivo?**

La función objetivo tiene como objetivo minimizar el costo total de grabar todas las escenas en los días, por lo que el costo total la suma del costo individual de cada día y el coste individual de cada día será la suma de los valores que cobran los actores por venir cada día de esta mora tenemos la siguiente función objetivo:

Minimizar $C = \sum_{d=1}^{D} \sum_{a=1}^{A} C_{a,d}$

Donde D es el número de días a firmar, A son los actores que tienen que firmar cada día y Cad es el valor que cobra cada actor por día.
Teniendo en cuenta que todos los actores cobran lo mismo y para comodidad sesteando como 1 el coste que tendría cada actor nos quedaría la siguiente función objetivo:

Minimizar $C = \sum_{d=1}^{D} A_d$

Donde A_d seria el numero de actores a presentarse a firmar en un día.

**¿Como implemento las restricciones?**

La restricción es que no se pueden hacer mas de 6 tomas por día por lo que para tenerla se implementaron las siguientes restricciones:

- Al generar una solución inicial se verifico que no se tomen mas de 6 tomas por día.

- Al generar los vecinos para el recocido simulado se implemento el cambio aleatorio de las tomas y así no sobrepasar el numero de tomas.


In [1]:
import random
import math

tomas = [
[1,1,1,1,1,0,0,0,0,0],
[0,0,1,1,1,0,0,0,0,0],
[0,1,0,0,1,0,1,0,0,0],
[1,1,0,0,0,0,1,1,0,0],
[0,1,0,1,0,0,0,1,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,1,1,0,0,0,0,0],
[1,1,0,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,0],
[1,1,0,0,0,1,0,0,1,0],
[1,1,1,0,1,0,0,1,0,0],
[1,1,1,1,0,1,0,0,0,0],
[1,0,0,1,1,0,0,0,0,0],
[1,0,1,0,0,1,0,0,0,0],
[1,1,0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0,0,1],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[1,0,1,1,1,0,0,0,0,0],
[0,0,0,0,0,1,0,1,0,0],
[1,1,1,1,0,0,0,0,0,0],
[1,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0],
[1,1,0,1,0,0,0,0,0,1],
[1,0,1,0,1,0,0,0,1,0],
[0,0,0,1,1,0,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0],
[1,0,0,0,1,1,0,0,0,0],
[1,0,0,1,0,0,0,0,0,0]
]

def convertir_a_diccionario(tomas):
    diccionario = {}
    for i, fila in enumerate(tomas):
        diccionario[f'T{i+1}'] = [f'A{j+1}' for j, valor in enumerate(fila) if valor == 1]
    return diccionario

def asignar_tomas_greedy(tomas):
    dias = []
    actores_por_dia = []
    tomas_pendientes = set(tomas.keys())

    while tomas_pendientes:
        dia = set()
        actores_del_dia = set()
        for _ in range(6):  # Máximo de 6 tomas por día
            mejor_toma = None
            max_comun = -1
            for toma in tomas_pendientes:
                actores_toma = set(tomas[toma])
                comun = len(actores_toma.intersection(actores_del_dia))
                if comun > max_comun:
                    mejor_toma = toma
                    max_comun = comun
            if mejor_toma:
                dia.add(mejor_toma)
                tomas_pendientes.remove(mejor_toma)
                actores_del_dia.update(tomas[mejor_toma])
            else:
                break  # No se encontraron más tomas óptimas para este día
        dias.append(dia)
        actores_por_dia.append(actores_del_dia)

    return dias, actores_por_dia


def calcular_costo(actores_por_dia):
  costo = 0
  for actores_del_dia in actores_por_dia:
      costo += len(actores_del_dia)
  return costo

def imprimir_solucion(dias, actores, costo):
  for i, (dia, actor) in enumerate(zip(dias, actores), 1):
    print(f"Día {i}: Tomas: {dia}, Actores: {actor}")
  print(f"Costo total de actores por día: {costo}")

def vecindad_intercambio(dias, max_tomas_por_dia=6):
    #Genera un nuevo estado intercambiando tomas entre dos días aleatorios.
    nuevo_estado = [set(dia) for dia in dias]
    dia1, dia2 = random.sample(range(len(nuevo_estado)), 2)
    if not nuevo_estado[dia1] or not nuevo_estado[dia2]:
        return nuevo_estado
    toma_dia1 = random.choice(list(nuevo_estado[dia1]))
    toma_dia2 = random.choice(list(nuevo_estado[dia2]))
    nuevo_estado[dia1].remove(toma_dia1)
    nuevo_estado[dia1].add(toma_dia2)
    nuevo_estado[dia2].remove(toma_dia2)
    nuevo_estado[dia2].add(toma_dia1)
    return nuevo_estado

def recocido_simulado(dias, actores_por_dia, temp_inicial, enfriamiento, iteraciones, max_tomas_por_dia=6):
    estado_actual = dias
    actores_estado_actual = actores_por_dia
    costo_actual = calcular_costo(actores_estado_actual)
    temperatura = temp_inicial

    for _ in range(iteraciones):
        nuevo_estado = vecindad_intercambio(estado_actual, max_tomas_por_dia)
        nuevos_actores_por_dia = [{actor for toma in dia for actor in tomas_dic[toma]} for dia in nuevo_estado]
        costo_nuevo = calcular_costo(nuevos_actores_por_dia)
        delta_costo = costo_nuevo - costo_actual
        if delta_costo < 0 or random.random() < math.exp(-delta_costo / temperatura):
            estado_actual = nuevo_estado
            actores_estado_actual = nuevos_actores_por_dia
            costo_actual = costo_nuevo
        temperatura *= enfriamiento
    return estado_actual, actores_estado_actual, costo_actual


def busqueda_local(dias, actores_por_dia, iteraciones, max_tomas_por_dia=6):
    estado_actual = dias
    actores_estado_actual = actores_por_dia
    costo_actual = calcular_costo(actores_estado_actual)
    for _ in range(iteraciones):
        encontrado_mejor = False
        for _ in range(100):
            nuevo_estado = vecindad_intercambio(estado_actual, max_tomas_por_dia)
            nuevos_actores_por_dia = [{actor for toma in dia for actor in tomas_dic[toma]} for dia in nuevo_estado]
            costo_nuevo = calcular_costo(nuevos_actores_por_dia)

            if costo_nuevo < costo_actual:
                estado_actual = nuevo_estado
                actores_estado_actual = nuevos_actores_por_dia
                costo_actual = costo_nuevo
                encontrado_mejor = True
                break
        if not encontrado_mejor:
            break
    return estado_actual, actores_estado_actual, costo_actual

tomas_dic = convertir_a_diccionario(tomas)
print(tomas_dic)

#Calculo del estado inicial Greedy
print("Estado inicial")
dias_de_grabacion_inicial, actores_de_grabacion_inicial = asignar_tomas_greedy(tomas_dic)
costo_inicial = calcular_costo(actores_de_grabacion_inicial)
imprimir_solucion(dias_de_grabacion_inicial, actores_de_grabacion_inicial, costo_inicial)

# Parámetros del Recocido Simulado
temp_inicial = 10
enfriamiento = 0.95
iteraciones = 1000

print("Recocido Simulado")

# Realizar la optimización con recocido simulado
dias_optimo_rs, actores_optimo_rs, costo_optimo_rs = recocido_simulado(dias_de_grabacion_inicial, actores_de_grabacion_inicial, temp_inicial, enfriamiento, iteraciones)
imprimir_solucion(dias_optimo_rs, actores_optimo_rs, costo_optimo_rs)


print("Recocido Simulado")

# Realizar la optimizacion con búsqueda local
estado_optimo_bl, actores_optimo_bl, costo_optimo_bl = busqueda_local(dias_de_grabacion_inicial, actores_de_grabacion_inicial, iteraciones)
imprimir_solucion(estado_optimo_bl, actores_optimo_bl, costo_optimo_bl)



{'T1': ['A1', 'A2', 'A3', 'A4', 'A5'], 'T2': ['A3', 'A4', 'A5'], 'T3': ['A2', 'A5', 'A7'], 'T4': ['A1', 'A2', 'A7', 'A8'], 'T5': ['A2', 'A4', 'A8'], 'T6': ['A1', 'A2', 'A4', 'A5'], 'T7': ['A1', 'A2', 'A4', 'A5'], 'T8': ['A1', 'A2', 'A6'], 'T9': ['A1', 'A2', 'A4'], 'T10': ['A1', 'A2', 'A6', 'A9'], 'T11': ['A1', 'A2', 'A3', 'A5', 'A8'], 'T12': ['A1', 'A2', 'A3', 'A4', 'A6'], 'T13': ['A1', 'A4', 'A5'], 'T14': ['A1', 'A3', 'A6'], 'T15': ['A1', 'A2', 'A7'], 'T16': ['A4', 'A10'], 'T17': ['A1', 'A3'], 'T18': ['A3', 'A6'], 'T19': ['A1', 'A3'], 'T20': ['A1', 'A3', 'A4', 'A5'], 'T21': ['A6', 'A8'], 'T22': ['A1', 'A2', 'A3', 'A4'], 'T23': ['A1', 'A3'], 'T24': ['A3', 'A6'], 'T25': ['A1', 'A2', 'A4', 'A10'], 'T26': ['A1', 'A3', 'A5', 'A9'], 'T27': ['A4', 'A5'], 'T28': ['A1', 'A4'], 'T29': ['A1', 'A5', 'A6'], 'T30': ['A1', 'A4']}
Estado inicial
Día 1: Tomas: {'T6', 'T14', 'T7', 'T22', 'T12', 'T1'}, Actores: {'A2', 'A1', 'A4', 'A6', 'A3', 'A5'}
Día 2: Tomas: {'T3', 'T11', 'T29', 'T4', 'T10', 'T26'}, 

#Análisis

**¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

En la complejidad del algoritmo intervienen los siguientes factores:
El algoritmo utilizado para calcular el estado inicial, el cual es la una sola iteración por lo que su complejidad computacional seria O(n).

Iteraciones del Algoritmo: La complejidad temporal principal proviene del número de iteraciones fijado para el algoritmo. Si realizamos N iteraciones, el costo temporal dependerá de lo que suceda en cada iteración.

Función de Vecindad: La operación de intercambio de tomas entre dos días se realiza en tiempo constante (O(1)) porque el acceso a los elementos de una lista y el intercambio son operaciones de tiempo constante. Sin embargo, el proceso de selección de las tomas y los días implica elegir elementos al azar, lo cual es también O(1).

Cálculo de Costos: Calcular el nuevo costo involucra iterar sobre cada día u contar el numero de actores que participan por día.

Teniendo en cuenta todos estos factores la complejidad final del algoritmo seria O(n).



#Diseño

**¿Que técnica utilizo? ¿Por qué?**

Para la solución de este modelo se utilizo un modelo de multiarrranque en el cual en la primera parte para la generación de una solución inicial se utilizó un algoritmo GREEDY el cual genera una situación inicial ya con la cantidad de tomas por día ideal, es decir de 6.

Luego se provo a hacer una búsqueda con cocido simulado y una búsqueda local, en esto se notó que la búsqueda con recocido simulado es mejor ya que esta evita estancarse en mínimos locales y obtiene mejores optimizaciones que la búsqueda local.

La función de vecindad a aplicar en estos 2 casos fue la de intercambiar tomas entre los días ya que con eso nos aseguramos de que nunca se pase de 6 tomas por día.

La combinación de estos algoritmos da un buen resultado que se prueba a continuación con otro conjunto de datos de entrada generados aleatoriamente.


In [5]:
def generar_matriz(m, n):
    matriz = []
    for _ in range(m):
        fila = []
        for _ in range(n):
            fila.append(random.randint(0, 1))
        matriz.append(fila)
    return matriz

m = 15
n = 15

matriz = generar_matriz(m, n)
print("Matriz generada:")
for fila in matriz:
    print(fila)


tomas_dic = convertir_a_diccionario(matriz)
print(tomas_dic)

#Calculo del estado inicial Greedy
print("Estado inicial")
dias_de_grabacion_inicial, actores_de_grabacion_inicial = asignar_tomas_greedy(tomas_dic)
costo_inicial = calcular_costo(actores_de_grabacion_inicial)
imprimir_solucion(dias_de_grabacion_inicial, actores_de_grabacion_inicial, costo_inicial)

# Parámetros del Recocido Simulado
temp_inicial = 10
enfriamiento = 0.95
iteraciones = 1000

print("Recocido Simulado")

# Realizar la optimización con recocido simulado
dias_optimo_rs, actores_optimo_rs, costo_optimo_rs = recocido_simulado(dias_de_grabacion_inicial, actores_de_grabacion_inicial, temp_inicial, enfriamiento, iteraciones)
imprimir_solucion(dias_optimo_rs, actores_optimo_rs, costo_optimo_rs)


print("Recocido Simulado")

# Realizar la optimizacion con búsqueda local
estado_optimo_bl, actores_optimo_bl, costo_optimo_bl = busqueda_local(dias_de_grabacion_inicial, actores_de_grabacion_inicial, iteraciones)
imprimir_solucion(estado_optimo_bl, actores_optimo_bl, costo_optimo_bl)


m = 50
n = 50

matriz = generar_matriz(m, n)
print("Matriz generada:")
for fila in matriz:
    print(fila)


tomas_dic = convertir_a_diccionario(matriz)
print(tomas_dic)

#Calculo del estado inicial Greedy
print("Estado inicial")
dias_de_grabacion_inicial, actores_de_grabacion_inicial = asignar_tomas_greedy(tomas_dic)
costo_inicial = calcular_costo(actores_de_grabacion_inicial)
imprimir_solucion(dias_de_grabacion_inicial, actores_de_grabacion_inicial, costo_inicial)

# Parámetros del Recocido Simulado
temp_inicial = 10
enfriamiento = 0.95
iteraciones = 1000

print("Recocido Simulado")

# Realizar la optimización con recocido simulado
dias_optimo_rs, actores_optimo_rs, costo_optimo_rs = recocido_simulado(dias_de_grabacion_inicial, actores_de_grabacion_inicial, temp_inicial, enfriamiento, iteraciones)
imprimir_solucion(dias_optimo_rs, actores_optimo_rs, costo_optimo_rs)


print("Recocido Simulado")

# Realizar la optimizacion con búsqueda local
estado_optimo_bl, actores_optimo_bl, costo_optimo_bl = busqueda_local(dias_de_grabacion_inicial, actores_de_grabacion_inicial, iteraciones)
imprimir_solucion(estado_optimo_bl, actores_optimo_bl, costo_optimo_bl)

Matriz generada:
[1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]
[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
{'T1': ['A1', 'A4', 'A5', 'A9', 'A11', 'A12', 'A13', 'A15'], 'T2': ['A4', 'A5', 'A6', 'A7', 'A10', 'A11', 'A13', 'A14'], 'T3': ['A2', 'A7', 'A8', 'A9', 'A10', 'A14', 'A15'], 'T4': ['A1', 'A2', 'A7', 'A10', 'A11', 'A13'], 'T5': ['A2', 'A3', 'A6', 'A7', 'A8', 'A9', 'A11', 'A14'], 'T6': ['A1', '